In [1]:
import skimage.io

import itertools
import sys

# Our numerical workhorses
import numpy as np
import pandas as pd
import scipy.integrate

# Import Altair for high level plotting
import altair as alt
import altair_catplot as altcat

# Import Bokeh modules for interactive plotting
import bokeh.io
import bokeh.plotting

import bebi103

# Set up Bokeh for inline viewing
bokeh.io.output_notebook()

# Pevent bulky altair plots
alt.data_transformers.enable('json')

# A whole bunch of skimage stuff
import skimage.feature
import skimage.filters
import skimage.filters.rank
import skimage.io
import skimage.morphology
import skimage.segmentation


/Users/maria/anaconda/lib/python3.6/site-packages/bebi103/viz.py:30: UserWarning: DataShader import failed with error "No module named 'datashader'".
Features requiring DataShader will not work and you will get exceptions.
  Features requiring DataShader will not work and you will get exceptions.""")
/Users/maria/anaconda/lib/python3.6/site-packages/bebi103/viz.py:38: UserWarning: Could not import `stan` submodule. Perhaps pystan is not properly installed.
  warnings.warn('Could not import `stan` submodule. Perhaps pystan is not properly installed.')
/Users/maria/anaconda/lib/python3.6/site-packages/bebi103/__init__.py:19: UserWarning: Could not import `stan` submodule. Perhaps pystan is not properly installed.
  warnings.warn('Could not import `stan` submodule. Perhaps pystan is not properly installed.')


Loading BokehJS ...

## Problem 3.2.b

The files bacterium_1.tif and bacterium_2.tif are TIFF stacks of time courses for specific cells in from the full field images. From these time courses, determine the bacterial area for each time point. Assign each time point with an identifier for which division it is. This means you need to determine when divisions happen.

We start by copying a function from tutorial 4b that allows us to display two images side by side. This will be helpful in checking the results of our segmentations.

In [4]:
def show_two_ims(im_1, im_2, titles=[None, None],
                 interpixel_distances=[0.065, 0.065], color_mapper=None):
    """Convenient function for showing two images side by side."""
    p_1 = bebi103.viz.imshow(im_1,
                             plot_height=300,
                             title=titles[0],
                             color_mapper=color_mapper,
                             interpixel_distance=interpixel_distances[0],
                             length_units='µm')
    p_2 = bebi103.viz.imshow(im_2,
                             plot_height=300,
                             title=titles[1],
                             color_mapper=color_mapper,
                             interpixel_distance=interpixel_distances[1],
                             length_units='µm')
    p_2.x_range = p_1.x_range
    p_2.y_range = p_1.y_range

    return bokeh.layouts.gridplot([p_1, p_2], ncols=2)

We load in the first bacterium.

In [5]:
# Define directory containing files
im_file = '../data/iyer_biswas_et_al/bacterium_1.tif'

# So we have it, the interpixel distance
ip = 0.052  # microns

# Get image file
im = skimage.io.imread(im_file)

We display the raw image of a random frame to see what we are looking at. We'll use it to perfect our segmentation pipeline, and then apply that segmentation pipeline to ALL the rest of the frame and the other bacterium.

In [24]:
# Display raw images
plot = bebi103.viz.imshow(im[180],
                            plot_height=300,
                            title=im_file,
                            interpixel_distance=ip,
                            length_units='µm')
bokeh.io.show(plot)

First, I tried using Canny Edge Detection. However, since we first find the edges and then fill the shapes in, we join the cells back together which isn't our final goal. The result is suseptible to noise. In our case, cells were not shaded in accurately enough for our needs, so we abondoned this method in favor of the Laplacian of Gaussian and zero crossing method.

In [25]:
# Convert image to float
im_float = (im[180].astype(float) - im[180].min()) / (im[180].max() - im[180].min())

# Do Canny edge detection on image, use sigma = 1.4
im_edge = skimage.feature.canny(im_float, 1.4)

# Show the result
# bokeh.io.show(bebi103.viz.imshow(im_edge))

bokeh.io.show(show_two_ims(im_float,
                           im_edge,
                           titles=['original', 'edges']))

Now we need to fill in the gaps in the edges!

In [26]:
# Fill the holes
im_bw = scipy.ndimage.morphology.binary_fill_holes(im_edge)

# Look at result
bokeh.io.show(show_two_ims(im_float,
                           im_bw,
                           titles=['original', 'segmented']))


In [27]:
# Close the edge image
selem = skimage.morphology.disk(2)
im_edge_closed = skimage.morphology.binary_closing(im_edge, selem)

# Fill these holes
im_bw = scipy.ndimage.morphology.binary_fill_holes(im_edge_closed)

# Check out results
bokeh.io.show(show_two_ims(im_float,
                           im_bw,
                           titles=['original', 'segmented']))

In [28]:
# Compute LoG
im_LoG = scipy.ndimage.filters.gaussian_laplace(im_float, 2.0)

# Check out results
bokeh.io.show(
    bebi103.viz.imshow(im_LoG,
                       color_mapper=bebi103.viz.mpl_cmap_to_color_mapper('bwr'),
                       interpixel_distance=ip,
                       length_units='µm',
                       colorbar=True))

In [29]:
# 3x3 square structuring element
selem = skimage.morphology.square(3)
    
# Do max filter and min filter
im_LoG_max = scipy.ndimage.filters.maximum_filter(im_LoG, footprint=selem)
im_LoG_min = scipy.ndimage.filters.minimum_filter(im_LoG, footprint=selem)

# Image of zero-crossings
im_edge = (  ((im_LoG >= 0) & (im_LoG_min < 0)) 
           | ((im_LoG <= 0) & (im_LoG_max > 0)))

# Show result
bokeh.io.show(show_two_ims(im_float,
                           im_edge,
                           titles=['original', 'edges']))

In [30]:
def zero_crossing_filter(im, thresh):
    """
    Returns image with 1 if there is a zero crossing and 0 otherwise.
    
    thresh is the the minimal value of the gradient, as computed by Sobel
    filter, at crossing to count as a crossing.
    """
    # Square structuring element
    selem = skimage.morphology.square(3)
    
    # Do max filter and min filter
    im_max = scipy.ndimage.filters.maximum_filter(im, footprint=selem)
    im_min = scipy.ndimage.filters.minimum_filter(im, footprint=selem)
    
    # Compute gradients using Sobel filter
    im_grad = skimage.filters.sobel(im)
    
    # Return edges
    return ( (  ((im >= 0) & (im_min < 0))
              | ((im <= 0) & (im_max > 0))) 
            & (im_grad >= thresh) )

In [31]:
# Find zero-crossings
im_edge = zero_crossing_filter(im_LoG, 0.001)

# Show result
bokeh.io.show(show_two_ims(im_float,
                           im_edge,
                           titles=['original', 'edges']))

In [32]:
# Skeletonize edges
im_edge = skimage.morphology.skeletonize(im_edge)

# See result
bokeh.io.show(show_two_ims(im_float,
                           im_edge,
                           titles=['original', 'edges']))

In [33]:
# Fill holes
im_bw = scipy.ndimage.morphology.binary_fill_holes(im_edge)

# Remove small objectes that are not bacteria
im_bw = skimage.morphology.remove_small_objects(im_bw, min_size=100)

# Show result
bokeh.io.show(show_two_ims(im_float,
                           im_bw,
                           titles=['original', 'segmented']))


In [43]:
# Clear border with large buffer size b/c LoG procedure came off border
im_bw = skimage.segmentation.clear_border(im_bw, buffer_size=5)

# Show result
bokeh.io.show(
    bebi103.viz.imshow(im_bw, 
                       color_mapper=bebi103.viz.mpl_cmap_to_color_mapper('gray'),
                       interpixel_distance=ip,
                       length_units='µm'))

In [42]:
# Label binary image; backward kwarg says value in im_bw to consider backgr.
im_labeled, n_labels = skimage.measure.label(
                            im_bw, background=0, return_num=True)

# Show number of bacteria
print('Number of individual bacteria = ', n_labels - 1)

print (im_labeled)

Number of individual bacteria =  0
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


Now that we have the number of bacteria in each image, we can start calculating the area at each frame. If the number of bacteria is greater than 1, we will need to filter out the extraneuos bacteria.

There are several options for doing this:

- pick the bacterium with the largest area
- pick the area nearest to the previous center (the main bacterium is stationary)
- pick the rightmost area (once again, the bacterium we want to isolate is stationary on the far right)

Once we have identified the bacterium, we will count the number of pixels to get the area and store it in an array.